In [273]:
import pandas as pd
import numpy as np
import random
import time

In [274]:
#df = pd.read_csv (r'C:\Notebook scripts\Features_Variant_1.csv')
df = pd.read_csv ('Features_Variant_1.csv')

data_train=np.asarray(df.iloc[:,:])
#print(np.shape(data_train))

#shuffle before folding
np.random.shuffle(data_train)

In [275]:
#fold_1, fold_2, fold_3, fold_4, fold_5= np.array_split(data_train,5)
data_train_arr=np.array_split(data_train,5) #{[],[],[],[],[]}

y_train_arr=[]
x_train_arr=[]
for i in range(len(data_train_arr)):
    y_train_arr.append(data_train_arr[i][:,-1].reshape((-1, 1)) )
    x_train_arr.append(data_train_arr[i][:,:-1])


In [276]:
def normalization(X):
    mean=np.mean(X,axis=0)#(1,53)
    std=np.std(X,axis=0)
    np.place(std, std==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_normalized=(X-mean)/std
    return x_normalized,mean,std
def normalization_with_params(X,mean,std):
    return (X-mean)/std

In [277]:
def linear_regression_predict(X,omega,omega0):
    prod=X.dot(omega)#(8190,1)
    #print(np.shape(y+prod))
    return prod+omega0

def root_mean_square_error(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    summands=np.power(y_actual-y_predict,2)
    #print(np.shape(summands))
    mse=np.sum(summands)/len(y_predict)
    return np.sqrt(mse)
   
def r_squared(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    nom=np.sum(np.power(y_actual-y_predict,2))
    denom=np.sum(np.power(y_actual-np.mean(y_actual,axis=0),2))
    return 1-nom/denom

In [278]:
def batch_gradient_descent(X,y_train,learning_rate,num_iterations):
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_iterations): 
        N=len(X)
        error=y_train-linear_regression_predict(X,omega,omega0)
        omega=omega+learning_rate*2*((X.T).dot(error))/N
        omega0=omega0+learning_rate*2*np.sum(error)/N
        cost[it] = r_squared(X,omega,omega0,y_train)
    return omega,omega0,cost

In [279]:
# function to create a list containing mini-batches 
def create_mini_batch(data,i, batch_size): 
    mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
    X_mini = mini_batch[:, :-1] 
    Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
       
    return X_mini,Y_mini

def mini_batch_gradient_descent(X,y_train,learning_rate,num_epoches,batch_size):
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
   
    it=0
    for it in range(num_epoches): 
        data = np.hstack((X, y_train)) 
        np.random.shuffle(data) #shuffle at each epoch (not every batch creation)
        n_minibatches = X.shape[0] // batch_size 
        i = 0
        for i in range(n_minibatches + 1):
            X_mini,y_mini= create_mini_batch(data,i, batch_size) 
        
            N=len(X_mini)
            error=y_mini-linear_regression_predict(X_mini,omega,omega0)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N
            omega0=omega0+learning_rate*2*np.sum(error)/N
            cost.append(r_squared(X_mini,omega,omega0, y_mini)) 
    return omega,omega0,cost

In [280]:
r2=[]
rmse=[]
omega=[]
omega0=[]
for item in range(len(x_train_arr)):
    x_test_rd=x_train_arr[item]
    y_test=y_train_arr[item]
    x_train_folds_arr=np.delete(x_train_arr,item)
    y_train_folds_arr=np.delete(y_train_arr,item)
    
    x_train_rd=np.empty((0,x_train_arr[item].shape[1]), int)
    y_train=np.empty((0,y_train_arr[item].shape[1]), int)
    #print(np.shape(y_train_folds_arr))
    for i in range (len(x_train_folds_arr)):
        #print(np.shape(y_train_folds_arr[i]))
        #x_train_all55 =(np.vstack((x_train_all55,x_train_folds_arr[i])))
        x_train_rd=np.concatenate((x_train_rd,x_train_arr[i]))
        y_train=np.concatenate((y_train,y_train_arr[i]))
    #print(np.shape(y_train))
    x_train,mean,std=normalization(x_train_rd)     #normalize all train dataset
    x_test=normalization_with_params(x_test_rd,mean,std)  #normalize test dataset with mean and std of train
    #print(x)
    #start = time.time()
    omega_item,omega0_item,cost=mini_batch_gradient_descent(x_train,y_train,learning_rate = 0.001,num_epoches=6,
                                                            batch_size=800)
    #stop = time.time()
    #duration = stop-start
    #print(duration)
    r2.insert(item,r_squared(x_test,omega_item,omega0_item,y_test))
    rmse.insert(item,root_mean_square_error(x_test,omega_item,omega0_item,y_test))
    omega.insert(item,omega_item)
    omega0.insert(item,omega0_item)
    print(r2[item],rmse[item])

0.2704429507696352 27.223257461730327
0.29310843948823606 32.44565607440142
0.24984232944135654 33.64935677862295
0.27037057744801707 28.281570037868182
0.24602397072592486 29.927637945290062


In [281]:

# initialise data of lists. 
data = {'1':[rmse[0],r2[0],omega0[0]], 
        '2':[rmse[1],r2[1],omega0[1]],
        '3':[rmse[2],r2[2],omega0[2]],
        '4':[rmse[3],r2[3],omega0[3]],
        '5':[rmse[4],r2[4],omega0[4]],
        'E':[np.mean(rmse),np.mean(r2), np.mean(omega0)],
        'SD':[np.std(rmse),np.std(r2), np.std(omega0)]} 
 
# Creates pandas DataFrame. 
df1 = pd.DataFrame(data, index =['RMSE', 'R^2', 'omega0']) 
df2 = pd.DataFrame(np.concatenate((np.hstack(omega),np.mean(omega,axis=0),np.std(omega,axis=0)),axis=1),
                   columns=['1', '2', '3','4','5','E','SD'])
df =pd.concat([df1, df2], axis=0)
df

,1,2,3,4,5,E,SD
RMSE,27.223257,32.445656,33.649357,28.281570,29.927638,30.305496,2.429030
R^2,0.270443,0.293108,0.249842,0.270371,0.246024,0.265958,0.016934
omega0,2.911396,2.872801,2.877412,2.842194,2.864221,2.873605,0.022441
0,0.143544,-0.037798,0.493894,0.476310,-0.023212,0.210548,0.233120
1,-0.153970,0.107427,-0.014819,-0.093252,0.061182,-0.018686,0.096083
2,0.233211,-0.019396,-0.022374,0.184260,-0.097703,0.055599,0.129078
3,0.057280,0.061300,0.618777,0.194342,0.685103,0.323360,0.273587
4,0.186674,0.508587,0.413646,-0.118185,0.098656,0.217876,0.224037
5,0.404581,0.635488,0.088458,0.559909,0.506019,0.438891,0.190674
6,0.569947,0.623356,0.846338,0.157190,0.212369,0.481840,0.260254
